<span style="font-size: 26px;">Importing libraries</span>

In [1]:
from IPython.display import Image
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import argparse
import sys
from pathlib import Path
import time

import cv2 as cv
import torch
import torch.backends.cudnn as cudnn

import subprocess
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk

from sort.sort import Sort
from PIL import Image
import regex as re

<span style="font-size: 26px;">PARAMETERS</span>

In [2]:
VIDEO_INPUT = ""
NAME = ""

<span style="font-size: 26px;">Detection functions</span>


In [3]:
# function to launch detect.py with the right parameters
def detect(WEIGHTS, CONF_TRESHOLD, DIRECTORY_TEST, IOU_THRES, NAME, visualize):
    print(f"Starting detection with weights {WEIGHTS}, conf_threshold {CONF_TRESHOLD}, directory_test {DIRECTORY_TEST}, iou_threshold {IOU_THRES}, and name {NAME}")

    # if visualize is checked, add it
    if visualize == 1:
        #command line to launch detect.py with parameters
        !python yolov5/detect.py --source {DIRECTORY_TEST} --weights {WEIGHTS} --conf {CONF_TRESHOLD} --save-txt --iou-thres {IOU_THRES} --visualize
        #command line with features
    else:
        !python yolov5/detect.py --source {DIRECTORY_TEST} --weights {WEIGHTS} --conf {CONF_TRESHOLD} --save-txt --iou-thres {IOU_THRES}

    # create a folder in Video_output with the NAME, if it exists, delete it and create a new one
    if os.path.exists(f"Video_output\\{NAME}"):
        shutil.rmtree(f"Video_output\\{NAME}")
        os.mkdir(f"Video_output\\{NAME}")
    else:
        os.mkdir(f"Video_output\\{NAME}")

    # move everything including folders in yolov5/runs/detect/exp folder to Video_output folder, if it's empty, wait 1 second and try again
    while True:
        try:
            for file in os.listdir("yolov5\\runs\\detect\\exp"):
                shutil.move(f"yolov5\\runs\\detect\\exp\\{file}", f"Video_output\\{NAME}")
            break
        except:
            time.sleep(1)
            continue

    # delete the empty folder
    os.rmdir("yolov5\\runs\\detect\\exp")
    print("Folder yolov5\\runs\\detect\\exp deleted")

    print("Detection done")

# function to find a file to detect
def open_file():
    global VIDEO_INPUT
    VIDEO_INPUT = filedialog.askopenfilename()
    print(VIDEO_INPUT)

# Start detection with inputs
def start_detection():
    # get values from inputs
    CONF_TRESHOLD = entry_conf.get()
    IOU_THRES = entry_iou.get()
    NAME = entry_name.get()
    visualize_var = int(visualize.get())

    # convert to float or default values
    if CONF_TRESHOLD == "":
        CONF_TRESHOLD = 0.4
    else:
        IOU_THRES = float(IOU_THRES)
    if IOU_THRES == "":
        IOU_THRES = 0.1
    else:
        CONF_TRESHOLD = float(CONF_TRESHOLD)
    if NAME == "":
        NAME = "Default"

    # path of our model weights
    WEIGHTS = 'yolov5\\runs\\train\\model\\weights\\best.pt'
    # launch detection
    detect(WEIGHTS, CONF_TRESHOLD, VIDEO_INPUT, IOU_THRES, NAME, visualize_var)
    return NAME

<span style="font-size: 26px;">Tracking</span>

In [5]:
def use_tracker(name):
    # Load the model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/model/weights/best.pt')

    # Change precision to float
    model.float()
    # Change to eval mode
    model.eval()

    # Paths
    savepath = 'Video_output/' + name + '/images_tracked/'
    # Create the folder if it doesn't exist
    if not os.path.exists(savepath):

        os.makedirs(savepath)
    print("Created folder: ", savepath)

    # Get the video path
    print( "video_input: " + VIDEO_INPUT)

    # get only the name of the video input and without .mp4
    vid_input_name = VIDEO_INPUT.split('/')
    vid_input_name = vid_input_name[-1]
    vid_input_name = vid_input_name.split('.')
    vid_input_name = vid_input_name[0]


    video_path = VIDEO_INPUT
    #keep only "Video_input/Video_name.mp4"
    video_path = video_path.split('/')
    video_path = video_path[-2] + '/' + video_path[-1]
    print("video_path: ", video_path)
    vid = cv.VideoCapture(video_path)
    det_path = 'Video_output/' + name + '/labels/'
    print("labels path: ", det_path)

    # tracker
    mot_tracker = Sort()
    # get number of frames
    print("Number of frames: ", int(vid.get(cv.CAP_PROP_FRAME_COUNT)))
    i=0
    while(vid.isOpened() and i < int(vid.get(cv.CAP_PROP_FRAME_COUNT))):
        # Read the video image by image
        ret, image_show = vid.read()
        print("Frame: ", i)
        # Do prediction on the image
        preds = model(image_show)

        # Get detections from the prediction
        detections2 = preds.pred[0].numpy()
        #print(detections2)
        print("")

        # Get width and height
        width = image_show.shape[1]
        height = image_show.shape[0]

        # Get the bounding boxes
        with open(det_path + vid_input_name +'_' + str(i+1) +'.txt', 'r') as f:
            lines = f.readlines()
        # array of bounding boxes
        bb = []
        # loop to get 4 values of the bounding box
        for line in lines:
            # Get values
            x_center, y_center, b_width, b_height = line.split(' ')[1:]
            # Get alues and multiply by width and height
            bb.append([float(x_center), float(y_center), float(b_width), float(b_height)])
            bb[-1][0] = bb[-1][0] * width
            bb[-1][1] = bb[-1][1] * height
            bb[-1][2] = bb[-1][2] * width
            bb[-1][3] = bb[-1][3] * height
            # the result of bb is an array of arrays of 4 elements, each element is a float corresponding to a coordinate in percentage of the frame, <x_center> <y_center> <width> <height>

        bb_in = []
        # This loop is used to set the bounding boxes in the right format for the tracker
        for b in bb:
            # Get values and set them in the right format
            x_min = int(b[0] - b[2]/2)
            y_min = int(b[1] - b[3]/2)
            width_b = int(b[0] + b[2]/2)
            height_b = int(b[1] + b[3]/2)
            # draw the bounding box
            #cv.rectangle(image_show, (x_min, y_min), (width_b, height_b), (255, 0, 255), 2)
            # write s in upper left corner
            #cv.putText(image_show, "s", (x_min, y_min), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            # draw line from upper left corner to bottom right corner
            #cv.line(image_show, (x_min, y_min), (x_min, height_b), (0, 255, 255), 2)
            #cv.line(image_show, (x_min, y_min), (width_b, y_min), (0, 255, 255), 2)

            # append x_min, y_min, width_b, height_b in bb_in
            bb_in.append([x_min, y_min, width_b, height_b])

        # Convert format to numpy
        bb = np.array(bb_in)
        # convert to numpy
        detections = np.array(bb)

        # update our tracker with new detections
        track_bbs_ids = mot_tracker.update(detections)
        # For each tracked object, draw the bounding box with the corresponding ID on the image
        for j in range(len(track_bbs_ids.tolist())):

            # Retrieve bounding box coordinates and object ID
            coords = track_bbs_ids.tolist()[j]
            x1,y1,x2,y2 = int(coords[0]),int(coords[1]),int(coords[2]),int(coords[3])
            name_idx = int(coords[4])

            # Create a string for the object ID
            name = "ID : " + str(name_idx)

            # Draw bounding box and ID on image
            cv.rectangle(image_show, (x1, y1), (x2, y2),(0, 255, 0) , 2)
            cv.putText(image_show, name, (x1, y1), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4)

            # Display image resized to 640*480
            cv.imshow('image_show', cv.resize(image_show, (640, 480)))
            #cv.imshow('image_show', image_show)

        # Register image with saved_path
        cv.imwrite(savepath + vid_input_name +'_' + str(i) +'.jpg', image_show)
        print("image saved: ", vid_input_name +'_' + str(i) +'.jpg')
        i += 1

        # Press 'x' to exit loop
        if cv.waitKey(1) & 0xFF == ord('x'):
            break

    # Release video and close all windows
    vid.release()
    cv.destroyAllWindows()

<span style="font-size: 26px;">Images to video converter</span>

In [4]:
def image_to_video(name):
    # path to the folder containing the images
    vid_path = 'Video_output/' + name + '/images_tracked/'

    # Name of the video (name + _detected)
    vid_name = 'Video_output/' + name + '/'+name+'_detected.mp4'

    # FPS of the video
    fps = 30

    # List of images, images are in order in folder
    img_files = os.listdir(vid_path)
    # order the images example name, ee_0,ee_10,ee12, order with digits
    img_files.sort(key=lambda f: int(re.sub('\D', '', f)))


    #print img_files name in order
    for i in range(len(img_files)):
        print(img_files[i])

    # assembler les images pour en faire une vidéo
    # Get the first image
    first_img = cv.imread(vid_path + img_files[0])
    # Get the height and width of the image
    height, width, layers = first_img.shape
    # Set the size of the video
    size = (width, height)
    # Create the video
    video = cv.VideoWriter(vid_name, cv.VideoWriter_fourcc(*'mp4v'), fps, size)

    # For each image in the list
    for img in img_files:
        # Get the image
        image = cv.imread(vid_path + img)
        # Add the image to the video
        video.write(image)

    # Close the video
    video.release()
    print(vid_path)
    print(vid_name)
    print('Vidéo créée avec succès :', vid_name)


<span style="font-size: 26px;">Tkinter app</span>

In [6]:
def launch():
    # Launch the detection
    name = start_detection()
    # Launch the video converter
    use_tracker(name)
    # Launch the video
    image_to_video(name)

In [ ]:
# create a window to choose the video to detect
root = tk.Tk()

# title of the window
root.title("Fish Detection")

# set the size and position of the window
window_width = 600
window_height = 400
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_pos = int(screen_width/2 - window_width/2)
y_pos = int(screen_height/2 - window_height/2)
root.geometry("{}x{}+{}+{}".format(window_width, window_height, x_pos, y_pos))

# set the window background color to a darker shade of gray
root.configure(bg='#222222')

# set the window transparency
root.attributes('-alpha', 0.9)

# create a label for the title
title_label = ttk.Label(root, text="Fish Tracker", font=("Helvetica", 18), background='#222222', foreground='white')
title_label.pack(pady=10)

# create a frame to contain the input fields
input_frame = ttk.Frame(root)
input_frame.pack(pady=10)

# button to choose the video to detect
button_file = ttk.Button(input_frame, text="Choose video to detect", command=open_file)
button_file.grid(row=0, column=0, padx=5, pady=5)

# input for the name of the output folder
text_name = ttk.Label(input_frame, text="Name of the output file default name : Default")
text_name.grid(row=1, column=0, padx=5, pady=5)
entry_name = ttk.Entry(input_frame)
entry_name.grid(row=1, column=1, padx=5, pady=5)

# input for the iou threshold and the confidence threshold
Iou_text = ttk.Label(input_frame, text="Iou threshold default : 0.1")
Iou_text.grid(row=2, column=0, padx=5, pady=5)
entry_iou = ttk.Entry(input_frame)
entry_iou.grid(row=2, column=1, padx=5, pady=5)

# input for the iou threshold and the confidence threshold
Conf_text = ttk.Label(input_frame, text="Confidence threshold default : 0.4")
Conf_text.grid(row=3, column=0, padx=5, pady=5)
entry_conf = ttk.Entry(input_frame)
entry_conf.grid(row=3, column=1, padx=5, pady=5)

# check button for visualize
visualize = tk.IntVar()
check = ttk.Checkbutton(input_frame, text="Visualize", variable=visualize)
check.grid(row=4, column=1, padx=5, pady=5)

# button to start detection
button_detection = ttk.Button(root, text="Start Detection", command=lambda: launch())
button_detection.pack(side=tk.RIGHT, padx=10, pady=10, anchor=tk.SE)

# center the input frame horizontally
input_frame.pack_propagate(0)
input_frame.place(relx=0.5, rely=0.4, anchor=tk.CENTER)

# loop to keep the window open
root.mainloop()

C:/Users/bouke/Desktop/Fish_tracking/Video_input/vid.mp4
Starting detection with weights yolov5\runs\train\model\weights\best.pt, conf_threshold 0.4, directory_test C:/Users/bouke/Desktop/Fish_tracking/Video_input/vid.mp4, iou_threshold 0.1, and name demo


detect: weights=['yolov5\\runs\\train\\model\\weights\\best.pt'], source=C:/Users/bouke/Desktop/Fish_tracking/Video_input/vid.mp4, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.1, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
fatal: Not a valid object name HEAD
YOLOv5  2022-3-18 torch 2.0.0+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients
video 1/1 (1/2416) C:\Users\bouke\Desktop\Fish_tracking\Video_input\vid.mp4: 480x640 11 Truites, Done. (0.762s)
video 1/1 (2/2416) C:\Users\bouke\Desktop\Fish_tracking\Video_input\vid.mp4: 480x640 10 Truites, Done. (0.573s)
video 1/1 (3/2416) C:\Users\bouke\Desktop\Fish_tracking\Video_input\vid.mp4: 480x640 10 Truites, Done

Folder yolov5\runs\detect\exp deleted
Detection done


Using cache found in C:\Users\bouke/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-4-14 Python-3.9.16 torch-2.0.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


Created folder:  Video_output/demo/images_tracked/
video_input: C:/Users/bouke/Desktop/Fish_tracking/Video_input/vid.mp4
video_path:  Video_input/vid.mp4
labels path:  Video_output/demo/labels/
Number of frames:  2416
Frame:  0

image saved:  vid_0.jpg
Frame:  1

image saved:  vid_1.jpg
Frame:  2

image saved:  vid_2.jpg
Frame:  3

image saved:  vid_3.jpg
Frame:  4

image saved:  vid_4.jpg
Frame:  5

image saved:  vid_5.jpg
Frame:  6

image saved:  vid_6.jpg
Frame:  7

image saved:  vid_7.jpg
Frame:  8

image saved:  vid_8.jpg
Frame:  9

image saved:  vid_9.jpg
Frame:  10

image saved:  vid_10.jpg
Frame:  11

image saved:  vid_11.jpg
Frame:  12

image saved:  vid_12.jpg
Frame:  13

image saved:  vid_13.jpg
Frame:  14

image saved:  vid_14.jpg
Frame:  15

image saved:  vid_15.jpg
Frame:  16

image saved:  vid_16.jpg
Frame:  17

image saved:  vid_17.jpg
Frame:  18

image saved:  vid_18.jpg
Frame:  19

image saved:  vid_19.jpg
Frame:  20

image saved:  vid_20.jpg
Frame:  21

image saved: 